<div align="right" style="text-align: right"><i>Peter Norvig<br>Dec 2018<br>Updated Jun 2020</i></div>

# Portmantout Words

A [***portmanteau***](https://en.wikipedia.org/wiki/Portmanteau) is a word that squishes together two words, like  *math* + *athlete* = *mathlete*.  Inspired by [**Darius Bacon**](http://wry.me/), I covered this as a programming exercise in my 2012 [**Udacity course**](https://www.udacity.com/course/design-of-computer-programs--cs212). In 2018 I was re-inspired by [**Tom Murphy VII**](http://tom7.org/), who added a new twist:  [***portmantout words***](http://www.cs.cmu.edu/~tom7/portmantout/) (*tout* from the French for *all*), which are defined as:

> A **portmantout** of a set of words $W$ is a string $S$ such that:
* Every word in $W$ is a **substring** of $S$. (Although a word may appear more than once in $S$.)
* The words **overlap**: the start of every word (except the first) is equal to or before the end of another word.
* **Nothing else** is in $S$: every letter in $S$ comes from the overlapping words. 

This notebook attempts to find a portmantout that is as short as possible, while covering a set $W$ of over 100,000 words. My program discovered:

- **preferendumdums**: a political commentary <br>portmanteau of {prefer, referendum, dumdums}
- **impromptutankhamenability**: a willingness to see the Egyptian exhibit on the spur of the moment<br>portmanteau of {impromptu, tutankhamen, amenability}
- **dashikimonogrammarianarchy**: chaos when a linguist enscribes African/Japanese garb<br>portmanteau of {dashiki, kimono, monogram, grammarian, anarchy}
- **allegestionstage**: a brutal theatre review<br>portmanteau of {alleges, egestions, onstage}
- **skymanipulablearsplittinglers**: a nerve-damaging aviator<br>portmanteau of {skyman, manipulable, blears, earsplitting, tinglers}
- **edinburgherselflesslylyricize**: a Scottish music review<br>portmanteau of {edinburgh, burghers, herself, selflessly, slyly, lyricize}
- **fortyphonshore**: a dire weather report<br>portmanteau of {forty, typhons, onshore} 


# Problem-Solving Strategy

Although I haven't proved it, my intuition is that finding a shortest $S$ is an NP-hard problem, and with 100,000 words to cover, it is unlikely that I can find an optimal solution in a reasonable amount of time. A common approach to NP-hard problems is a **greedy algorithm**: make the locally best choice at each step, in the hope that the steps will fit together into a solution that is not too far from the globally shortest solution. 

Thus, my approach will be to build up a **path**, starting with one word, and then adding **steps**  to the end of the evolving path, one at a time. Each step consists of a word from *W* that overlaps the end of the previous word by at least one letter. I will choose the step that seems to be the best choice at the time (the one with the **lowest cost**), and will never undo a step, even if the path seems to get stuck later on. I distinguish two types of steps:

- **Unused word step**: using a word for the first time. (Once we use them all, we're done.)
- **Bridging word step**: if no remaining unused words overlap the previous word, we need to do something to get back on track to consuming unused words. I call that something a **bridge**: a step that **repeats** a previously-used word and has an ending that matches the start of some unused word. Repeating a word may add letters to the final length of $S$, and it doesn't get us closer to the requirement of using all the words, but it is sometimes necessary. Sometimes two words are required to build a bridge, but never more than two (at least with the 100,000 word set we will be dealing with). 

There's actually a third type of word, but it doesn't need a corresponding type of step: a **subword** is a word that is a substring of another word. If, say, `ajar` is in $W$, then we know we have to place it in some step along the path. But if `jar` is also in $W$, we don't need a separate step for it&mdash;whenever we place `ajar`, we have automatically placed `jar`. The program can save computation time by initializing the set of **unused words** to be just the **nonsubwords** in $W$.  (We are still free to use a subword as a **bridging word** if needed.)

(*English trivia:* I use the clumsy term "nonsubwords" rather than "superwords", because there are a small number of words, like "cozy" and "pugs" and "july," that are not subwords and also not superwords.)

If we're going to be **greedy**, we need to know what we're being greedy about. I will always choose a step that minimizes the **number of excess letters** that the step adds. To be precise: I arbitrarily assume a baseline model in which all the words are concatenated with no repeated words and no overlap between them. (That's not a valid solution, but it is useful as a benchmark.) So if I add an unused word, and overlap it with the previous word by three letters, that is an excess of -3 (a negative excess is a positive thing): I've saved three letters over just concatenating the unused word. For a bridging word, the excess is the number of letters that do not overlap either the previous word or the next word. 

**Examples:** In each row of the table below, `'ajar'` is the previous word,  but each row makes different assumptions about what unused words remain, and thus we get different choices for the step to take. The table shows the overlapping letters between the previous word and the step, and in the case of bridges, it shows the next unused word that the step is bridging to. In the final column we give the excess letters (if any) and excess score.

|Previous|Step(s)|Overlap|Bridge to|Type of Step|Excess|
|--------|----|----|---|---|---|
| ajar|jarring|jar||unused word |-3 |
| ajar|arbitrary|ar||unused word |-2|
| ajar|argot|ar|goths|one-step bridge |0|
| ajar|arrow|ar|owlets| one-step bridge|"r" 1|
| ajar|rani+iraq|r|quizzed| two-step bridge|"anira" 5 | 

Let's go over the example steps one row at a time:
- **jarring**: Here we assume `jarring` is an unused word, and it overlaps by 3 letters, giving it an excess cost of -3, which is the best possible (an overlap of 4 would mean `ajar` is a subword, and we already agreed to eliminate subwords).
- **arbitrary**: an unused word that overlaps by 2 letters, so it would only be chosen if there were no unused words with 3-letter overlap.
- **argot**: From here on down, we assume there are no unused words that overlap, which means that `argot` is a best-possible bridge, because it completely overlaps the `ar` in `ajar` and the `got` in an unused word, `goths`.
- **arrow**: overlaps the `ar` in `ajar` and the `ow` in an unused word, `owlets`, but that leaves one excess letter, `r`, for an excess cost of 1.
- **rani + iraq**: Suppose `quizzed` is the only remaining unused word. There is no single word that bridges from any suffix of `ajar` to any prefix of `quizzed`. But I have arranged to build two-word bridges for every combination of one letter on the left and one letter on the right (unless there was already a shorter one-word bridge for that combination). The bridge from `'r'` to `'q'` is  `rani` followed by `iraq`, which has an excess score of 5 due to the letters `anira` not overlapping anything.

We see that unused word steps always have a negative excess cost (that's good) while bridge steps always have a zero or positive excess cost; thus an unused word step is always better than a bridge step (according to this metric).




# Data Type Implementation

Here I describe how to implement the main data types in Python:

- **Wordset**: a subclass of `set`, denoting a set of words.
- **Word**: a Python `str` (as are subparts of words, like suffixes or individual letters).
- **Step**: a namedtuple of an overlap and a word; the step adding `jarring` to `ajar` would be `Step(3, 'jarring')`. The first step in a path should have an overlap of 0; all others should have a positive integer overlap.
- **Path**: a Python `list` of steps.
- **Bridge**: a namedtuple of an excess cost followed by a list of steps, e.g. `Bridge(1, [Step(2, 'arrow')])`.


(*Python trivia:* I implemented `Wordset` as a subclass of `set` so that I can add attributes (such as `W.subwords`) to it later. You can do that with a user-defined subclass, but not with a builtin class.)

In [1]:
from collections import defaultdict, Counter, namedtuple
from typing      import List, Tuple, Set, Dict, Any
from statistics  import mean

In [2]:
class Wordset(set): """A set of words."""

Word   = str
Step   = namedtuple('Step', 'overlap, word')
Path   = List[Step]
Bridge = namedtuple('Bridge', 'excess, steps')

# 108,709 Word Set: W 

We can make Tom Murphy's 108,709-word list `"wordlist.asc"` into a `Wordset`,  $W$:

In [3]:
! [ -e wordlist.asc ] || curl -O https://norvig.com/ngrams/wordlist.asc

In [4]:
W = Wordset(open('wordlist.asc').read().split())
len(W)

108709

# Overall Program Design

I  thought I would define a major function, `portman`, to generate the portmantout string $S$ from the set of words $W$ according to the strategy outlined above, and a minor function, `is_portman`, to verify the result. But I found verification to be difficult. For example, given  $S =$  `'helloworld'` I would reject that as non-overlapping if I parsed it as `'hello'` + `'world'`, but I would accept it if parsed as `'hello'` + `'low'` + `'world'`. It was hard for `is_portman` to decide which parse was intended, which is a shame because `portman` *knew* which was intended, but  discarded the information. 

Therefore, I decided to change the interface: I'll have one function that takes $W$ as input and returns  a path $P$, and a second function to generate the string $S$ from $P$.  I decided on the following calling and [naming](https://en.wikipedia.org/wiki/Natalie_Portman) conventions:

    P = natalie(W)    # Find a portmantout path P for a set of words W
    is_portman(P, W)  # Verify whether P is a valid path covering words W
    S = portman(P)    # Compute the string S from the path P

Thus I can generate a string $S$ with:

    S = portman(natalie(W))
    
# portman

Here is the definition of `portman` and the result of `portman(P1)`, where the path `P1` covers the word set `W1` (which has five nonsubwords and 16 subwords):

In [5]:
def portman(P: Path) -> Word:
    """Compute the portmantout string S from the path P."""
    return ''.join(word[overlap:] for (overlap, word) in P)

In [6]:
W1 = Wordset('''anarchy dashiki grammarian kimono monogram an am arc arch i
                aria as ash dash gram grammar mar narc no on ram'''.split())
P1 = [Step(0, 'dashiki'),
      Step(2,      'kimono'),
      Step(4,        'monogram'),
      Step(4,            'grammarian'),
      Step(2 ,                   'anarchy')]
S1 =          'dashikimonogrammarianarchy'

In [7]:
portman(P1)

'dashikimonogrammarianarchy'

# natalie

As stated above, the approach is to start with one word (either given as an optional argument to `natalie` or chosen arbitrarily from the word set $W$), and then repeatedly add steps, each step being either an unused word or a bridging word. In order to make this process efficient, we arrange for `precompute(W)` to cache the following information in `W`:
  - `W.subwords`: a set of all the words that are contained within another word in `W`.
  - `W.bridges`: a dict where `W.bridges[suf][pre]` gives the best bridge between the two word affixes.
  - `W.unused`: initially the set of nonsubwords in `W`; when a word is used it is removed from the set.
  - `W.startswith`: a dict that maps from a prefix to all the unused words that start with the prefix. A word is removed from all the places it appears when it is used. Example: `W.startswith['somet'] == {'something', 'sometimes'}`.
  
These structures are somewhat complicated, so don't be discouraged if you have to go over a line of code or a prose description word-by-word several times before you understand exactly how it works.

After the precomputation, `natalie` loops until there are no more unused words. On each turn we call `unused_step`, which returns a list of one step if an unused word overlaps, or the empty list if it doesn't, in which case we call `bridging_steps`, which always returns a bridge of either one or two steps. We then append the step(s) to the path `P`, and call `mark_as_used(W, word)` to mark that `word` has been used (reducing the size of `W.unused` and updating `W.startswith` if `word` was previously unused). 

It is important that every bridge leads to an unused word. That way we know the program will **always terminate**: if  $N$ is the number of unused nonsubwords in $W$, then consider the quantity $(2N + (1$ `if` the last step overlaps an unused word `else` $0))$. Every iteration of the `while` loop decreases this quantity by at least 1; therefore the quantity will eventually be zero, and when it is zero, it must be that `W.unused` is empty and the loop terminates.

In [8]:
def natalie(W: Wordset, start=None) -> Path:
    """Return a portmantout path containing all words in W."""
    precompute(W)
    word = start or next(iter(W.unused))
    mark_as_used(W, word)
    P = [Step(0, word)]
    while W.unused:
        for step in unused_step(W, word) or bridging_steps(W, word):
            P.append(step)
            word = step.word
            mark_as_used(W, word)
    return P

`unused_step` considers every suffix of the previous word, longest suffix first. If a suffix starts any unused words, we choose the first such word as the step. Since we're going longest-suffix first, no other word could do better, and since unused steps have negative costs and bridges don't, the unused step will always be better than any bridge.

`bridging_steps` also tries every suffix of the previous word, and for each one it looks in the `W.bridges[suf]` table to see what prefixes (of unused words) we can bridge to from this suffix. Consider all such `W.bridges[suf][pre]` entries that bridge to the prefix of an unused word (as maintained in `W.startswith[pre]`). Out of all such bridges, take one with the minimal excess cost, and return the one- or two-step sequence that makes up the bridge.

So, both `unused_steps` and `bridging_steps` return a list of steps, the former either zero or one steps; the latter either one or two steps.

In [9]:
def unused_step(W: Wordset, prev_word: Word) -> List[Step]:
    """Return [Step(overlap, unused_word)] or []."""
    for suf in suffixes(prev_word):
        for word in W.startswith.get(suf, ()):
            return [Step(len(suf), word)]
    return []

def bridging_steps(W: Wordset, prev_word: Word) -> List[Step]:
    """The steps from the bridge with minimum excesss that bridges 
    from a suffix of prev_word to a prefix of an unused word."""
    bridge = min(W.bridges[suf][pre] 
                 for suf in suffixes(prev_word) if suf in W.bridges
                 for pre in W.bridges[suf] if W.startswith[pre])
    return bridge.steps

**Failure is not an option**: what happens if we have a small word set that can't make a portmantout? First `unused_step` will fail to find an unused word, which is fine; then `bridging_steps` will fail to find a bridge, and will raise `ValueError: min() arg is an empty sequence`. You could catch that error and return, say, an empty path if you wanted to, but my intended use is for word sets where this can never happen.

# precompute etc.

Here are the subfunctions that make the code above work:

In [10]:
def precompute(W):
    """Precompute and cache data structures for W. The .subwords and .bridges
    data structures are static and only need to be computed once; .unused and
    .startswith are dynamic and must be recomputed on each call to `natalie`."""
    if not hasattr(W, 'subwords') or not hasattr(W, 'bridges'): 
        W.subwords = subwords(W)
        W.bridges  = build_bridges(W)
    W.unused       = W - W.subwords
    W.startswith   = compute_startswith(W.unused)
    
def mark_as_used(W, word):
    """Record the fact that `word` has been used.
    Remove word from `W.unused` and from `W.startswith[pre]`, for each prefix."""
    if word in W.unused:
        W.unused.remove(word)
        for pre in prefixes(word):
            W.startswith[pre].remove(word)
            if not W.startswith[pre]:
                del W.startswith[pre]
                
def multimap(pairs) -> Dict[Any, set]:
    """Given (key, val) pairs, make a dict of {key: {val,...}}."""
    result = defaultdict(set)
    for key, val in pairs:
        result[key].add(val)
    return result

def compute_startswith(words) -> Dict[str, Set[Word]]: 
    """A dict mapping a prefix to all the words it starts:
    {'somet': {'something', 'sometimes'},...}."""
    return multimap((pre, w) for w in words for pre in prefixes(w))

def subwords(W: Wordset) -> Set[str]:
    """All the words in W that are subparts of some other word."""
    return {subword for w in W for subword in subparts(w) & W}              
    
def suffixes(word) -> List[str]:
    """All non-empty proper suffixes of word, longest first."""
    return [word[i:] for i in range(1, len(word))]

def prefixes(word) -> List[str]:
    """All non-empty proper prefixes of word."""
    return [word[:i] for i in range(1, len(word))]

def subparts(word) -> Set[str]:
    """All non-empty proper substrings of word"""
    return {word[i:j] 
            for i in range(len(word)) 
            for j in range(i + 1, len(word) + (i > 0))}

(*Python trivia:* the `(i > 0)` in the last line of  `subparts` means that for a four-letter word like `'gram'`, we include subparts `word[i:4]` except when `i == 0`, thus including `'ram'`, `'am'`, and `'m'`, but not `'gram'`. In Python it is considered good style to have a Boolean expression like `(i > 0)` automatically converted to an integer `0` or `1`.)

(*Math trivia:* "Proper" means "not whole". A proper subset is a subset that is not the whole set itself; a proper subpart of a word is a part (i.e. substring) of the word that is not the whole word itself.)

# Building Bridges


The last piece of the program is the construction of the `W.bridges` table. Recall that we want `W.bridges[suf][pre]` to be a bridge between a suffix of the previous word and a prefix of some new word, as in these examples:

      W.bridges['ar']['ow'] == Bridge(1, [Step(2, 'arrow')])
      W.bridges['ar']['c']  == Bridge(0, [Step(2, 'arc')])
      W.bridges['r']['q']   == Bridge(5, [Step(1, 'rani'), Step(1, 'iraq')])
      
We build all the bridges once and for all in `precompute`, and don't update them as words are used. Thus, `W.bridges['ar']['c']` says "if there are any unused words starting with `'c'`, you can use this bridge, but I'm not promising there are any." The caller (i.e. `bridging_steps`) is responsible for checking that `W.startswith['c']` contains unused word(s).
      
Bridges should be short. We don't need to consider `antidisestablishmentarianism` as a possible bridge word. Instead, from our 108,709  word set $W$, we'll select the words with length up to 5 (10,273 words), plus six-letter words that end in any of the rarest letters, 'qujvz' (20 words). For other word sets, you may have to modify these parameters. I call these `shortwords`. I also compute a `shortstartswith` table for the `shortwords`, where, for example,

     shortstartswith['som'] == {'soma', 'somas', 'some'} # but not 'somersaulting', ...
     
To build one-word bridges, consider every shortword, and split it up in all possible ways into a prefix that will overlap the previous word, a suffix that will overlap the next word, and a count of zero or more excess letters in the middle that don't overlap anything. For example:

In [11]:
def splits(word) -> List[Tuple[int, str, str]]: 
    """A sequence of (excess, pre, suf) tuples."""
    return [(excess, word[:i], word[i+excess:])
            for excess in range(len(word) - 1)
            for i in range(1, len(word) - excess)]

In [12]:
splits('arrow')

[(0, 'a', 'rrow'),
 (0, 'ar', 'row'),
 (0, 'arr', 'ow'),
 (0, 'arro', 'w'),
 (1, 'a', 'row'),
 (1, 'ar', 'ow'),
 (1, 'arr', 'w'),
 (2, 'a', 'ow'),
 (2, 'ar', 'w'),
 (3, 'a', 'w')]

The first element of the list says that `'arrow'` can bridge from `'a'` to `'rrow'` with 0 excess letters; the last says it can bridge from `'a'` to `'w'` with 3 excess letters (which happen to be `'rro'`). We consider every possible split, and pass it on to `try_bridge`, which records the bridge in the table under `bridges[pre][suf]` unless there is already a shorter bridge stored there.

In [13]:
def try_bridge(bridges, pre, suf, excess, word, step2=None):
    """Store a new bridge if it has less excess than the previous bridges[pre][suf]."""
    if pre != suf and (suf not in bridges[pre] or excess < bridges[pre][suf].excess):
        steps = [Step(len(pre), word)]
        if step2: steps.append(step2)
        bridges[pre][suf] = Bridge(excess, steps)

Now for two-word bridges. I thought that if I allowed all possible two-word bridges the program would be slow because there would be so many of them, and most of them would be too long to be of any use. Thus, I decided to only use two-word bridges that bridge from the last letter in the previous word to the first letter in an unused word.

We start out the same way, looking at every shortword. But this time we look at every suffix of each shortword, and see if the suffix starts another shortword. If it does, then we have a two-word bridge. Here's the complete `build_bridges` function:

In [14]:
def build_bridges(W: Wordset, maxlen=5, end='qujvz'):
    """A table of bridges[pre][suf] == Bridge(excess, steps), e.g.
    bridges['ar']['c'] == Bridge(0, [Step(2, 'arc')])."""
    bridges         = defaultdict(dict)
    shortwords      = [w for w in W if len(w) <= maxlen + (w[-1] in end)]
    shortstartswith = compute_startswith(shortwords)
    # One-word bridges
    for word in shortwords: 
        for excess, pre, suf, in splits(word):
            try_bridge(bridges, pre, suf, excess, word)
    # Two-word bridges
    for word1 in shortwords:
        for suf in suffixes(word1): 
            for word2 in shortstartswith[suf]: 
                excess = len(word1) + len(word2) - len(suf) - 2
                A, B = word1[0], word2[-1]
                step2 = Step(len(suf), word2)
                try_bridge(bridges, A, B, excess, word1, step2)
    return bridges

(*Note:* When we are *using* a bridge, we say `W.bridges[suf][pre]` but when we are *building* the bridge we say `W.bridges[pre][suf]`. It may seem confusing to write the code both ways, but that's because the very definition of **overlapping** is that the suffix of one word is the same as the prefix of the next, so it just depends how you are looking at it.)

**Missing bridges:** How do we know if a word set will always generate a portmantout? If the word set has a bridge from every one-letter suffix to every one other one-letter prefix, then it will always be able to bridge from any word to any unused word.  (Of course, there are other conditions under which it can succeed.) The function `missing_bridges` tells us which of these one-letter-to-one-letter bridges are missing:

In [15]:
def missing_bridges(W):
    """What one-letter-to-one-letter bridges are missing from W.bridges?"""
    return {A + B for A in alphabet for B in alphabet 
            if A != B and B not in W.bridges[A]}

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [16]:
precompute(W)
missing_bridges(W)

set()

Great! $W$ has no missing bridges. But the tiny word set $W1$ is missing 630 out of a possible 650 bridges:

In [17]:
precompute(W1)
len(missing_bridges(W1)), 26 * 25

(630, 650)

# Portmantout for W1

**Finally!** We're ready to make portmantouts. First for the small word list `W1`:

In [18]:
natalie(W1, start='dashiki')

[Step(overlap=0, word='dashiki'),
 Step(overlap=2, word='kimono'),
 Step(overlap=4, word='monogram'),
 Step(overlap=4, word='grammarian'),
 Step(overlap=2, word='anarchy')]

In [19]:
portman(natalie(W1, start='dashiki'))

'dashikimonogrammarianarchy'

# Portmantout for W

Now the portmantout for the full word set $W$:

In [20]:
%time P = natalie(W)
len(P)

CPU times: user 7.81 s, sys: 48.9 ms, total: 7.86 s
Wall time: 7.89 s


103088

In [21]:
S = portman(P)
len(S)

553761

I thought it might take 10 minutes, so under 10 seconds is great. But I'm not really excited to look at half a million letters in $S$; I'll define a better way to explore the portmantout.

# Making it Prettier

I'll define `report` to print various statistics, summarize the start and end of the portmantout, and save $S$ into [a file](natalie.txt). 

In [22]:
def report(W, P, steps=50, letters=500, save='natalie.txt'):
    '''Print information about the portmantout and optionally `save` to file.'''
    S       = portman(P)
    sub     = W.subwords
    nonsub  = W - sub
    bridge  = len(P) - len(nonsub)
    missing = len(missing_bridges(W)) or "no"
    valid   = "is" if is_portman(P, W) else "IS NOT"
    def L(words): return sum(map(len, words)) # Number of letters
    print(f'''
    W has {len(W):,d} words ({len(sub):,d} subwords; {len(nonsub):,d} nonsubwords)
    P has {len(P):,d} steps ({bridge:,d} bridge words)
    S has {len(S):,d} letters; W has {L(W):,d}; nonsubs have {L(nonsub):,d}
    P has an average overlap of {(L(w for _,w in P)-len(S))/(len(P)-1):.2f} letters
    S has a compression ratio (letters(W)/letters(S)) of {L(W)/len(S):.2f}
    P (and thus S) {valid} a valid portmantout of W
    W has {missing} missing one-letter-to-one-letter bridges''')
    if save:
        print(f'    S saved as "{save}", ({open(save, "w").write(S)} bytes)')
    print(f'\nThe first and last {steps} steps are:\n')
    for step in (*P[:steps], Step('.', '...'), *P[-steps:]):
        print(step.overlap, step.word)
    print(f'\nThe first and last {letters} letters are:\n\n{S[:letters]} ... {S[-letters:]}')

def is_portman(P: Path, W: Wordset) -> str:
    """Verify that P forms a valid portmantout string for W."""
    all_words = (W - W.subwords) <= set(w.word for w in P) <= W
    overlaps  = all(overlap > 0 and P[i - 1][1][-overlap:] == word[:overlap]
                    for i, (overlap, word) in enumerate(P[1:], 1))
    return all_words and overlaps and P[0].overlap == 0 

(*Python trivia:* if `X, Y` and `Z` are sets, `X <= Y <= Z` means "is `X` a subset of `Y` and `Y` a subset of `Z`?" We use the notation here to say that the set of words in $P$ must contain all the nonsubwords and can only contain words from $W$.)

In [23]:
report(W, P)


    W has 108,709 words (44,320 subwords; 64,389 nonsubwords)
    P has 103,088 steps (38,699 bridge words)
    S has 553,761 letters; W has 931,823; nonsubs have 595,805
    P has an average overlap of 1.64 letters
    S has a compression ratio (letters(W)/letters(S)) of 1.68
    P (and thus S) is a valid portmantout of W
    W has no missing one-letter-to-one-letter bridges
    S saved as "natalie.txt", (553761 bytes)

The first and last 50 steps are:

0 leadworks
5 workstations
3 onstage
5 stagey
3 geysers
3 ersatzes
3 zesting
5 stingier
2 eroding
4 dinginess
5 inessential
9 essentially
4 allyls
1 straightest
4 testings
1 solderers
3 erstwhile
5 whiles
3 lessening
3 ingressive
2 vending
4 dinghy
2 hygienics
1 snooted
3 tediums
1 sandpile
4 pileups
3 upshots
4 hotspurs
4 pursed
3 seducible
3 blending
4 dingles
3 lessees
4 seesawed
3 wedgier
2 erupted
3 teddy
4 eddying
5 dyings
1 suffocated
3 tediously
3 slyest
4 yesteryears
4 earshots
4 hotshots
2 tsunamis
4 amiss
4 misspoke
5 spoke

# Exploring

The program is complete, but there are still many interesting things to explore. 

**Question**: is there an imbalance in starting and ending letters of words? 

This is important because it could lead to a need for many two-word bridges. We saw that the last 50 steps of $P$ all involved words that start with `q`, or bridges to them. What  letters besides `q` might lead to problems?

In [24]:
precompute(W)
words  = W.unused # The nonsub words in W

In [25]:
# How many nonsubwords start with each letter?
Counter(w[0] for w in words).most_common() 

[('s', 7388),
 ('c', 5849),
 ('p', 4977),
 ('d', 4093),
 ('r', 3811),
 ('b', 3776),
 ('a', 3528),
 ('m', 3405),
 ('t', 3097),
 ('f', 2794),
 ('i', 2771),
 ('u', 2557),
 ('e', 2470),
 ('g', 2177),
 ('h', 2169),
 ('o', 1797),
 ('l', 1634),
 ('w', 1561),
 ('n', 1542),
 ('v', 1032),
 ('j', 638),
 ('k', 566),
 ('q', 330),
 ('y', 207),
 ('z', 169),
 ('x', 51)]

In [26]:
# How many nonsubwords end with each letter?
Counter(w[-1] for w in words).most_common() 

[('s', 29056),
 ('y', 8086),
 ('d', 7520),
 ('g', 6343),
 ('e', 3215),
 ('t', 2107),
 ('r', 1994),
 ('n', 1860),
 ('l', 1182),
 ('c', 908),
 ('m', 657),
 ('a', 384),
 ('h', 351),
 ('k', 157),
 ('i', 128),
 ('p', 123),
 ('o', 113),
 ('x', 68),
 ('f', 51),
 ('w', 42),
 ('z', 21),
 ('u', 11),
 ('b', 6),
 ('v', 6)]

Yes, there is a problem: there are 330 words that start with `q` and no nonsubwords that end in `q` (there are two subwords, `colloq` and `iraq`).  Here's a comparison of how many times a `q` appears in the first three letters and last three  letters  of nonsubwords:

In [27]:
qstart = {w for w in words if 'q' in w[:3]}
len(qstart)

710

In [28]:
qend = {w for w in words if 'q' in w[-3:]}
len(qend)

23

In [29]:
' '.join(qend)

'albuquerque statuesque diplomatique semiopaque mozambique romanesque odalisque colloquy cliquy risque maqui picaresque perique manque prosequi iraqis bezique soliloquy obloquy ventriloquy placque obsequy hdqrs'

There is also a problem with 29,056 nonsubwords ending in `s` and only 7,388 starting with `s`. But many words start with combinations like `as` or `es` or `ps`, so the overall ratio is not as bad:

In [30]:
sstart = {w for w in words if 's' in w[:3]}
len(sstart)

12247

In [31]:
send = {w for w in words if 's' in w[-3:]}
len(send)

32394

**Question**: what are the most common steps in $P$? 

These will be bridge words, which can be repeated any number of times. What do they have in common?

In [32]:
Counter(P).most_common(25)

[(Step(overlap=1, word='sac'), 3170),
 (Step(overlap=1, word='so'), 2079),
 (Step(overlap=2, word='lyre'), 1649),
 (Step(overlap=1, word='dab'), 1628),
 (Step(overlap=1, word='of'), 1591),
 (Step(overlap=1, word='sun'), 1487),
 (Step(overlap=1, word='gab'), 1483),
 (Step(overlap=1, word='sin'), 1407),
 (Step(overlap=1, word='sip'), 1062),
 (Step(overlap=1, word='sam'), 971),
 (Step(overlap=1, word='yam'), 958),
 (Step(overlap=1, word='sew'), 789),
 (Step(overlap=2, word='lye'), 707),
 (Step(overlap=1, word='spa'), 612),
 (Step(overlap=1, word='gun'), 503),
 (Step(overlap=3, word='erst'), 489),
 (Step(overlap=1, word='yen'), 466),
 (Step(overlap=2, word='type'), 450),
 (Step(overlap=1, word='gip'), 437),
 (Step(overlap=1, word='econ'), 423),
 (Step(overlap=1, word='she'), 392),
 (Step(overlap=1, word='go'), 389),
 (Step(overlap=1, word='sex'), 356),
 (Step(overlap=1, word='yep'), 323),
 (Step(overlap=1, word='you'), 321)]

Even though `iraq` dominated the last 50 steps, that's not true of `P` overall. Instead, it looks like bridging away from `s` is a big concern (as expected by the letter counts). (Also, `lyre` and `lye` bridge from an adverb ending.) The following says that about 30% of all steps in `P` bridge from a suffix that contains `s`:

In [33]:
sum('s' in w[:o] for o, w in P) / len(P)

0.2984246469036163

**Question:** What is the distribution of word lengths? 

In [34]:
Counter(sorted(map(len, words))) # Counter of word lengths

Counter({3: 2,
         4: 186,
         5: 1796,
         6: 4364,
         7: 8672,
         8: 11964,
         9: 11950,
         10: 8443,
         11: 6093,
         12: 4423,
         13: 2885,
         14: 1765,
         15: 1017,
         16: 469,
         17: 198,
         18: 91,
         19: 33,
         20: 22,
         21: 9,
         22: 4,
         23: 2,
         28: 1})

**Question:** What is the longest nonsubword? The shortest?

In [35]:
max(words, key=len)

'antidisestablishmentarianism'

In [36]:
{w for w in words if len(w) <= 3}

{'fbi', 'ibm'}

**Question**: What is the distribution of letters?

In [37]:
Counter(L for w in words for L in w).most_common() # Counter of letters

[('e', 68038),
 ('s', 60080),
 ('i', 53340),
 ('a', 43177),
 ('n', 42145),
 ('r', 41794),
 ('t', 38093),
 ('o', 35027),
 ('l', 32356),
 ('c', 23100),
 ('d', 22448),
 ('u', 19898),
 ('g', 17815),
 ('p', 16128),
 ('m', 16062),
 ('h', 12673),
 ('y', 11889),
 ('b', 11581),
 ('f', 7885),
 ('v', 5982),
 ('k', 4892),
 ('w', 4880),
 ('z', 2703),
 ('x', 1677),
 ('j', 1076),
 ('q', 1066)]

**Question**: How many bridges are there? How many excess letters do they have? How many steps?

Note this is across all the bridges that exist, without regard for whether (or how often) they are used in `P`.

In [38]:
# A list of all bridges
B = sorted((W.bridges[suf][pre], pre, suf) 
           for suf in W.bridges for pre in W.bridges[suf])
len(B)

56430

In [39]:
# Counter of bridge excess letters
Counter(b.excess for (b, *_) in B)

Counter({0: 37172, 1: 16689, 2: 2417, 3: 92, 4: 32, 5: 21, 6: 6, 8: 1})

In [40]:
# Average excess
mean(_.elements())

0.39121034910508595

In [41]:
# Number of steps
Counter(len(b.steps) for (b, *_) in B)

Counter({1: 56280, 2: 150})

I'm surprised there are only 150 two-step bridges.

**Question**: What do some bridges look like?

In [42]:
B[::1000] # Sample every 1000th bridge

[(Bridge(excess=0, steps=[Step(overlap=1, word='aahed')]), 'ahed', 'a'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='bra')]), 'ra', 'b'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='darn')]), 'arn', 'd'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='fines')]), 'ines', 'f'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='heman')]), 'eman', 'h'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='left')]), 'eft', 'l'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='newel')]), 'ewel', 'n'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='puffy')]), 'uffy', 'p'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='shoes')]), 'hoes', 's'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='thug')]), 'hug', 't'),
 (Bridge(excess=0, steps=[Step(overlap=1, word='winy')]), 'iny', 'w'),
 (Bridge(excess=0, steps=[Step(overlap=2, word='bests')]), 'sts', 'be'),
 (Bridge(excess=0, steps=[Step(overlap=2, word='cored')]), 'red', 'co'),
 (Bridge(excess=0, steps=[Step(overlap=2, word='etc')]), 'c', '

**Question**: What strange letter combinations are there? 

Let's look at two-letter suffixes or prefixes that only appear in one or two nonsubwords. 

In [43]:
{pre: W.startswith[pre] # Rare two-letter prefixes
 for pre in sorted(W.startswith) 
 if len(pre) == 2 and 1 <= len(W.startswith[pre]) <= 2}

{'aj': {'ajar'},
 'ay': {'ayahs', 'ayatollahs'},
 'bw': {'bwanas'},
 'ct': {'ctrl'},
 'dn': {'dnieper'},
 'dv': {'dvorak'},
 'ee': {'eelgrasses', 'eelworm'},
 'ek': {'ekistics'},
 'ez': {'ezekiel'},
 'fb': {'fbi'},
 'fj': {'fjords'},
 'gj': {'gjetosts'},
 'gw': {'gweducks', 'gweducs'},
 'hd': {'hdqrs'},
 'ie': {'ieee'},
 'if': {'iffiness'},
 'ik': {'ikebanas', 'ikons'},
 'ip': {'ipecacs'},
 'iv': {'ivories', 'ivory'},
 'jn': {'jnanas'},
 'kw': {'kwachas', 'kwashiorkor'},
 'll': {'llamas', 'llanos'},
 'mc': {'mcdonald'},
 'oj': {'ojibwas'},
 'pf': {'pfennigs'},
 'qa': {'qaids', 'qatar'},
 'qo': {'qophs'},
 'sf': {'sforzatos'},
 'tc': {'tchaikovsky'},
 'uf': {'ufos'},
 'wu': {'wurzel'},
 'xi': {'xiphoids', 'xiphosuran'},
 'xm': {'xmases'},
 'yc': {'ycleped', 'yclept'},
 'ym': {'ymca'},
 'zl': {'zlotys'},
 'zw': {'zwiebacks'}}

In [44]:
endswith = multimap((w[-2:], w) for w in words)

{suf: endswith[suf] # Rare two-letter suffixes
 for suf in sorted(endswith) 
 if 1 <= len(endswith[suf]) <= 2}

{'aa': {'markkaa'},
 'ab': {'skylab'},
 'ai': {'bonsai'},
 'ao': {'chiao', 'ciao'},
 'bm': {'ibm', 'icbm'},
 'cd': {'recd'},
 'dn': {'haydn'},
 'dt': {'rembrandt'},
 'ec': {'filespec', 'quebec'},
 'ef': {'unicef'},
 'eh': {'mikveh', 'yahweh'},
 'ep': {'asleep', 'shlep'},
 'fa': {'khalifa'},
 'gm': {'apophthegm'},
 'gn': {'champaign'},
 'hm': {'microhm'},
 'hn': {'mendelssohn'},
 'ho': {'groucho'},
 'hr': {'kieselguhr'},
 'hu': {'buchu'},
 'ji': {'fiji'},
 'ko': {'gingko', 'stinko'},
 'ku': {'haiku'},
 'lm': {'stockholm', 'unhelm'},
 'ln': {'lincoln'},
 'lu': {'honolulu'},
 'mb': {'clomb', 'whitecomb'},
 'mp': {'prestamp'},
 'mt': {'daydreamt', 'undreamt'},
 'nc': {'dezinc', 'quidnunc'},
 'nu': {'vishnu'},
 'nx': {'bronx', 'meninx'},
 'nz': {'franz'},
 'ob': {'blowjob'},
 'oe': {'monroe'},
 'oi': {'hanoi', 'polloi'},
 'ou': {'thankyou'},
 'oz': {'kolkhoz'},
 'pa': {'tampa'},
 'po': {'troppo'},
 'rb': {'cowherb'},
 'rf': {'waldorf', 'windsurf'},
 'ru': {'nehru'},
 'sr': {'ussr'},
 'su': 

The two-letter prefixes definitely include some strange words.

The list of two-letter suffixes is mostly pointing out flaws in the word list. For example, lots of words end in `ab`: blab, cab, dab, gab, etc. But must of them are subwords (of blabs, cabs, dabs, gabs, etc.); only `skylab` made it into the word list in singular form but not plural.

# Comparison to Tom Murphy's Program

To compare my program to [Murphy's](https://sourceforge.net/p/tom7misc/svn/HEAD/tree/trunk/portmantout/): I used a greedy approach that incrementally builds up a single long portmanteau, extending it via a bridge when necessary. Murphy first built a pool of smaller portmanteaux, then joined them all together. I'm reminded of the [Traveling Salesperson Problem](TSP.ipynb) where one algorithm is to form a single path, always progressing to the nearest neighbor, and another algorithm is to maintain a pool of shorter segments and repeatedly join together the two closest segments. The two approaches are different, but it is not clear whether one is definitively better than the other (either for TSP or for portmantout). You could try it!

(*English trivia:*  my program builds a single path of words, and when the path gets stuck and I need something to allow me to continue, it makes sense to call that thing a **bridge**.  Murphy's program starts by building a large pool of small portmanteaux that he calls **particles**, and when he can build no more particles, his next step is to put two particles together, so he calls it a **join**. The different metaphors for what our programs are doing lead to different terminology for the same idea.)

In terms of implementation, mine is in Python and written at a high level of abstraction; Murphy's is in C++ and is at a lower level. That's one reason why his code is 10 times longer; another reason is that he does a lot of cool extra work with generating diagrams and animations. 

It appears Murphy didn't quite have the complete concept of **subwords**. He did mention that when he adds `'bulleting'`, he crosses `'bullet'` and `'bulletin'` off the list, but somehow  [his string](http://tom7.org/portmantout/murphy2015portmantout.pdf) contains both `'spectacular'` and `'spectaculars'` in two different places. My guess is that when he adds `'spectaculars'` he crosses off `'spectacular'`, but if he happens to add `'spectacular'` first, he will later add `'spectaculars'`. Support for this guess is that his output in `bench.txt` says "I skipped 24319 words that were already substrs";  there are 44,320 such subwords, and he found about half of them. I think those missing 20,001 words are the main reason why my $S$ strings are coming in at around 554,000 letters, about 57,000 letters shorter than Murphy's best case of 611,820 letters.

Also, Murphy's *joins* are always between one-letter prefixes and suffixes. I do the same thing for two-word bridges, because having a `W.bridges[A][B]` for every letter `A` and `B` is  the easiest way to prove that the program will terminate. But for one-word bridges, I allow prefixes and suffixes of any length up to a total of 6 for `len(pre) + len(suf)`. I can get away with this because I limited my candidate pool to the 10,000 `shortwords`. It would have been untenable to build all bridges of all lengths for all 100,000 words, and probably would not have helped shorten $S$ appreciably.

I should say that I stole a trick from Murphy. I started watching his highly-entertaining [video](https://www.youtube.com/watch?time_continue=1&v=QVn2PZGZxaI), but then I paused it because I wanted the fun of solving the problem mostly on my own. After I finished the first version of my program, I returned to the video and [paper](http://tom7.org/portmantout/murphy2015portmantout.pdf) and I noticed that I had a problem in my use of bridges. My program originally looked something like this: 

    step = unused_step(...) or one_word_bridge(...) or two_word_bridge(...)
    
That is, I only considered two-word bridges when there was no one-word bridge, on the theory that one word is shorter than two. But Murphy showed that my theory was wrong: I had `'workaholic'`, a one-word bridge, for `bridges['w']['c']`, but Murphy had the two-word bridge  `'war' + 'arc' = 'warc'`, which saves six letters. After seeing that, I shamelessly copied his approach, and now I too get a four-letter `bridges['w']['c']`:

In [45]:
W.bridges['w']['c']

Bridge(excess=2, steps=[Step(overlap=1, word='we'), Step(overlap=1, word='etc')])

# Conclusion

I'll stop here, but you should feel free to do more experimentation of your own. 

Here are some things you could do to make the portmantouts more interesting:

- Use linguistic resources (such as [pretrained word embeddings](https://nlp.stanford.edu/projects/glove/)) to teach your program what words are related to each other. Encourage the program to place  related words next to each other.
- Use linguistic resources (such as [NLTK](https://github.com/nltk/)) to teach your program where syllable breaks are in words, and what each syllable sounds like. Encourage the program to make overlaps match syllables. (That's why "preferendumdums" sounds better than "fortyphonshore".)

Here are some things you could do to make $S$ shorter:

- **Lookahead**: Unused words are chosen based on the degree of overlap, but nothing else. It might help to prefer unused words which have a suffix that matches the prefix of another unused word. A single-word lookahead or a bbeam search could be used.

- **Reserving words**: It seems like `haydn` and `dnieper` are made to go together; they're the only words with `dn` as an affix. If `haydn` was selected first, `unused_step` would select `dnieper` next, but if `dnieper` as selected first, we'd lose the chance to take advantage of that overlap. Maybe there could be a system that assures `haydn` comes first, or a preprocessing step that joined together words that uniquely go together. This is getting close to what Murphy did in his program.

- **Word choice ordering**: Perhaps `compute_startswith` could sort the words in each key's bucket so that the "difficult" words (say, the ones that end in unusual letters) are encountered earlier in the program's execution, when there are more available words for them to connect to.
  
Here are some things you could do to make the program more robust:

- Write and run unit tests. (I have some cells that informally test things, but no formal tests.)

- Find other word lists, perhaps in other languages, and try the program on them.

- Consider what to do for a wordset that has missing bridges. You could try three-word bridges, you could allow the program to back up and remove a previously-placed word; you could allow the addition of words to the start as well as the end of `P`.